# Denoising Autoencoder

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Helpers/NeuralNetworkBase.ipynb");

In [ ]:
train_alpha(
    create_hyperparams(
        "autoencoder",
        "explicit",
        ["ExplicitUserItemBiases"],
        ["ExplicitUserItemBiases"],
    ),
    "NeuralExplicitAutoencoderUntuned";
    tune_hyperparams=false
)

In [ ]:
train_alpha(
    create_hyperparams(
        "autoencoder",
        "implicit",
        ["NeuralImplicitUserItemBiases"],
        ["ExplicitUserItemBiases"],
    ),
    "NeuralImplicitAutoencoder";
    tune_hyperparams=false    
)

In [ ]:
train_alpha(
    create_hyperparams(
        "autoencoder",
        "ptw",
        ["NeuralPtwUserItemBiases"],
        ["ExplicitUserItemBiases"],
    ),
    "NeuralPtwAutoencoder";
    tune_hyperparams=false
)